## Training plan:
Using LLama Llama-3.2-1B-bnb-4bit (not instruct model)
we train 2 variations:

We train one epoch with 94.5 of the data with 5% val 0.5% test. We also include our new data. : https://huggingface.co/datasets/mlfoundations-dev/stackexchange_mythology/viewer/default/train?p=6 ?

Where 90% train, 10 % test

For our own data, we dupplicate training data

- One with new data same hyperparams
- One with new data and better hyperparam



In [1]:
%%capture
%pip install unsloth
# Also get the latest nightly Unsloth!
#%uv pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

In [2]:
import torch
from dataclasses import dataclass
from unsloth import FastLanguageModel
from peft import PeftModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
TOKEN = None

import torch
from dataclasses import dataclass
from unsloth import FastLanguageModel
from peft import PeftModel

MODEL_REPOS = {
    "original": "unsloth/Llama-3.2-1B-bnb-4bit",
    "vanilla":  "StefanCoder1/Scalable-vanilla",
    "tuned":    "StefanCoder1/Scalable-tuned",
    "qwen":     "StefanCoder1/Granite-tuned",
}

@dataclass
class ModelSlot:
    model_type: str
    model: any = None
    tokenizer: any = None

MODELS = {
    "original": ModelSlot("original"),
    "vanilla":  ModelSlot("vanilla"),
    "tuned":    ModelSlot("tuned"),
    "qwen":     ModelSlot("qwen"),
}

def load_new(
    llama = True,
    model_type="tuned",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True
):
    
    if llama:
        repo = MODEL_REPOS[model_type]

        # Load base HF/Unsloth model
        org_model, org_tokenizer = FastLanguageModel.from_pretrained(
            model_name = repo,
            max_seq_length = max_seq_length,
            dtype = dtype,
            load_in_4bit = load_in_4bit,
            token = TOKEN
        )

        # Original model is returned directly (no merging)
        if model_type == "original":
            return org_model, org_tokenizer

        # For vanilla/tuned → load LoRA adapter and merge
        adapter_repo = MODEL_REPOS[model_type]
        model = PeftModel.from_pretrained(
            org_model,
            adapter_repo,
            token = TOKEN,
        )

        merged = model.merge_and_unload()
        return merged, org_tokenizer
    else: 
        # for the qwen model: 
        org_qwen_model, org_qwen_tokenizer = FastLanguageModel.from_pretrained(
            model_name = "unsloth/Qwen2-0.5B-bnb-4bit",
            max_seq_length = max_seq_length,
            dtype = dtype,
            load_in_4bit = load_in_4bit,
            token = TOKEN
        )
        adapter_repo = MODEL_REPOS[model_type]
        model = PeftModel.from_pretrained(
            org_qwen_model,
            adapter_repo,
            token = TOKEN,
        )
        merged = model.merge_and_unload()
        return merged, org_qwen_tokenizer


for key in ["original", "vanilla", "tuned", "qwen"]:
    print(f"Loading {key}...")
    if key == "qwen":
        m, t = load_new(llama=False, model_type=key)
    else:
        m, t = load_new(key)
    MODELS[key].model = m
    MODELS[key].tokenizer = t
    print(f"{key} loaded ✓")

Loading original...
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/35.2M [00:00<?, ?B/s]

qwen loaded ✓


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
from datasets import load_dataset
from datasets import concatenate_datasets

from unsloth.chat_templates import standardize_sharegpt
from unsloth.chat_templates import get_chat_template

tokenizer_llama = get_chat_template(
    MODELS["original"].tokenizer,
    chat_template = "llama-3.2", 
)

tokenizer_qwen = get_chat_template(
    MODELS["qwen"].tokenizer,
    chat_template = "qwen-2.5",
)


def create_ds(llama = True):
    
    def formatting_prompts_func(examples):
        if llama:
            tokenizer = tokenizer_llama
        else:
            tokenizer = tokenizer_qwen
        

        convos = examples["conversations"]
        texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
        return { "text" : texts, }
    


    dataset2 = load_dataset("mlfoundations-dev/stackexchange_mythology", split="train")
    dataset2 = dataset2.select_columns(["conversations"])
    dataset2 = standardize_sharegpt(dataset2)
    dataset2 = dataset2.map(formatting_prompts_func, batched = True)

    dataset = load_dataset("mlabonne/FineTome-100k", split = "train")
    dataset = standardize_sharegpt(dataset)
    dataset = dataset.map(formatting_prompts_func, batched = True)

    train_testvalid = dataset.train_test_split(test_size=0.055, seed=3407)
    test_valid = train_testvalid['test'].train_test_split(test_size=1/11, seed=3407)
    test_dataset = test_valid['test']

    train_testvalid2 = dataset2.train_test_split(test_size=0.1, seed=3407)
    test_dataset2 = train_testvalid2['test']

    merged_test = concatenate_datasets([test_dataset, test_dataset2])
    print(f"Test size: {len(merged_test)}")
    merged_test = merged_test.shuffle(seed=3407)
    return merged_test

merged_test = create_ds(llama=True)
merged_test_qwen = create_ds(llama=False)

In [5]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

def eval_causal_lm(model, tokenizer, test_dataset,
                   batch_size=4, max_len=2048, device=None):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    model.eval()

    def collate_fn(batch):
        texts = [ex["text"] for ex in batch]   # <-- adapt key if needed
        toks = tokenizer(
            texts,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=max_len,
        )
        toks["labels"] = toks["input_ids"].clone()
        return {k: v.to(device) for k, v in toks.items()}

    loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn,
    )

    total_loss = 0.0
    total_tokens = 0
    print("len(test_dataset) =", len(test_dataset))
    with torch.no_grad():
        for i, batch in enumerate(tqdm(loader)):
            outputs = model(**batch)
            loss = outputs.loss
            n_tokens = batch["labels"].numel()
            total_loss += loss.item() * n_tokens
            total_tokens += n_tokens
    print(total_tokens)
    avg_loss = total_loss / total_tokens
    ppl = torch.exp(torch.tensor(avg_loss))

    return avg_loss, ppl.item()


#original_loss, original_ppl =eval_causal_lm(MODELS["original"].model,   MODELS["original"].tokenizer,   merged_test)
#print(f"loss {original_loss} ppl {original_ppl}")

qwen_loss, qwen_ppl =eval_causal_lm(MODELS["qwen"].model,   MODELS["qwen"].tokenizer,   merged_test_qwen)
print(f"loss {qwen_loss} ppl {qwen_ppl}")


len(test_dataset) = 1031


100%|██████████| 258/258 [03:15<00:00,  1.32it/s]

974289
loss 6.64315115772972 ppl 767.5098266601562


In [40]:
vanillia_loss, vanillia_ppl =eval_causal_lm(MODELS["vanilla"].model, MODELS["vanilla"].tokenizer, merged_test)
print(f"loss {vanillia_loss} ppl {vanillia_ppl}")


len(test_dataset) = 1031


100%|████████████████████████████████████████████████████████████████████████| 258/258 [04:28<00:00,  1.04s/it]

972085
loss 8.371501560724571 ppl 4322.12255859375


In [41]:
tuned_loss, tuned_ppl =eval_causal_lm(MODELS["tuned"].model, MODELS["tuned"].tokenizer, merged_test)
print(f"loss {tuned_loss} ppl {tuned_ppl}")

len(test_dataset) = 1031


100%|████████████████████████████████████████████████████████████████████████| 258/258 [04:28<00:00,  1.04s/it]

972085
loss 6.931065466342436 ppl 1023.5841064453125
